In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
import glob
import os
import pickle
import json

import cv2
import numpy as np

In [3]:
from skimage.feature import hog

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, f1_score 
from sklearn.metrics import accuracy_score, make_scorer

In [4]:
DATA_DIR = os.path.join('../data')
VEHICLES = os.path.join(DATA_DIR, 'vehicles')
NON_VEHICLES = os.path.join(DATA_DIR, 'non-vehicles')

In [5]:
def hog_feature(image, orientations=9):
    return hog(image, orientations=orientations, 
               pixels_per_cell=(8, 8), 
               cells_per_block=(2, 2), 
               feature_vector=True)
    
def feature_extractor(image, orientations=9):
    img = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
    # HOG Features
    rhf = hog_feature(img[:, :, 0], orientations)
    ghf = hog_feature(img[:, :, 1], orientations)
    bhf = hog_feature(img[:, :, 2], orientations)
    return np.hstack((rhf, ghf, bhf))

def feature_extractor_w_hist(image, orientations=9):
    img = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
    # HOG Features
    rhf = hog_feature(img[:, :, 0], orientations)
    ghf = hog_feature(img[:, :, 1], orientations)
    bhf = hog_feature(img[:, :, 2], orientations)
    # Histogram Features
    channel1_hist = np.histogram(img[:,:,0], bins=32)
    channel2_hist = np.histogram(img[:,:,1], bins=32)
    channel3_hist = np.histogram(img[:,:,2], bins=32)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    return np.hstack((rhf, ghf, bhf, hist_features))

In [6]:
def read(data_dir, fn):
    for img_path in glob.iglob(os.path.join(data_dir, '*', '*.png')):
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)/255
        yield fn(image)

In [13]:
vehicles = [feature for feature in read(VEHICLES, feature_extractor_w_hist)]
non_vehicles = [feature for feature in read(NON_VEHICLES, feature_extractor_w_hist)]

In [14]:
data = np.vstack((vehicles, non_vehicles))
labels = np.hstack((np.ones(len(vehicles), np.int32), np.zeros(len(non_vehicles), np.int32)))

In [15]:
data.shape

(17760, 5388)

In [16]:
data, labels = shuffle(data, labels)
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [17]:
pipe1 = Pipeline([('scaler', StandardScaler()), ('svc', svm.LinearSVC())])
pipe1.fit(x_train, y_train)

pred = pipe1.predict(x_test)
cm = confusion_matrix(pred, y_test)
acc = accuracy_score(pred, y_test)
f1 = f1_score(pred, y_test)
print(acc, cm, f1)

0.986204954955 [[1734   25]
 [  24 1769]] 0.98633955952


In [18]:
with open('classifier_hist.p', 'wb') as _file:
    pickle.dump(pipe1, _file)